In [1]:
##########################################################################
# Created on Sat Nov 01 23:21:45 2021                                    #
# Python for Financial Analysis and Risk Management                      #
# @author: Meng Lipeng (FRM, CFA)                                        #
##########################################################################

# 6.6.1.Measure FX forward rate

In [12]:
def FX_forward(spot,r_A,r_B,T):
    '''Define a function for FX forward calculation.
    spot: spot FX rate. A/B
    r_A: risk free rate for currency A. annual compound
    r_B: risk free rate for currency B. annual compound
    T: Tenor for FX forward in year'''
    forward=spot*(1+r_A*T)/(1+r_B*T)
    return forward

IR      | 1M       | 3M      | 6M       | 1Y  
:------:| :----:   | :----:  | :----:   | :----:  
Shibor  | 1.5820%  | 1.5940% | 1.6680%  | 1.9030% 
Libor   | 0.1801%  | 0.3129% | 0.4813%  | 0.6340% 

In [13]:
import numpy as np

FX_spot=7.0965
Tenor=np.array([1/12,3/12,6/12,1.0])
Shibor=np.array([0.015820,0.015940,0.016680,0.019030])
Libor=np.array([0.001801,0.003129,0.004813,0.006340])

FX_forward_list=np.zeros_like(Tenor)
for i in range(len(Tenor)):
    FX_forward_list[i]=FX_forward(spot=FX_spot,r_A=Shibor[i],r_B=Libor[i],T=Tenor[i])
   
print('one month FX forward rate(CNY/USD) ',round(FX_forward_list[0],4))
print('three month FX forward rate(CNY/USD) ',round(FX_forward_list[1],4))
print('six month FX forward rate(CNY/USD) ',round(FX_forward_list[2],4))
print('one year FX forward rate(CNY/USD) ',round(FX_forward_list[3],4))

one month FX forward rate(CNY/USD)  7.1048
three month FX forward rate(CNY/USD)  7.1192
six month FX forward rate(CNY/USD)  7.1385
one year FX forward rate(CNY/USD)  7.186


# 6.6.2.Covered interst arbitrage

In [14]:
def cov_arbitrage(S,F,M_A,M_B,r_A,r_B,T,A,B):
    '''Define a function for calculating covered interest arbitrage profit and show the sequence for arbitrage
    S: spot FX rate. A/B
    F: forward FX rate in market.
    M_A: notional amount of A. 'Na' indicates not known yet
    M_B: notional amount of B. 'Na' indicates not known yet
    r_A: risk free rate of A. annual compound
    r_B: risk free rate of B. annual compound
    T: Tenor for FX forward in year
    A: currency name of A
    B: currency name of B'''
    F_new=S*(1+r_A*T)/(1+r_B*T)
    if F_new<F:
        if M_B=='Na':
            profit=M_A*(1+T*r_B)*F/S-M_A*(1+T*r_A)
            if profit>0:
                sequence=['arbitrage sequence: ',
                         '(1)Name of currency borrowed at the initial time ',A,
                         '(2)Name of currency converted and invested at spot exchange rate ',B,
                         '(3)Name of currency converted at the end of the investment at the forward exchange rate',A,
                         '(4)Repayment of borrowed funds at the initial time.']
            else:
                sequence=['No arbitrage opportunity exists.']
        else:
            profit=M_B*S*(1+T*r_A)/F-M_B*(1+T*r_B)
            if profit>0:
                sequence=['arbitrage sequence: ',
                         '(1)Name of currency borrowed at the initial time ',B,
                         '(2)Name of currency converted and invested at spot exchange rate ',A,
                         '(3)Name of currency converted at the end of the investment at the forward exchange rate',B,
                         '(4)Repayment of borrowed funds at the initial time.']
            else:
                sequence=['No arbitrage opportunity exists.']
    elif F_new>F:
        if M_B=='Na':
            profit=M_A*(1+T*r_B)*F/S-M_A*(1+T*r_A)
            if profit>0:
                sequence=['arbitrage sequence: ',
                         '(1)Name of currency borrowed at the initial time ',A,
                         '(2)Name of currency converted and invested at spot exchange rate ',B,
                         '(3)Name of currency converted at the end of the investment at the forward exchange rate',A,
                         '(4)Repayment of borrowed funds at the initial time.']
            else:
                sequence=['No arbitrage opportunity exists.']
        else:
            profit=M_B*S*(1+T*r_A)/F-M_B*(1+T*r_B)
            if profit>0:
                sequence=['arbitrage sequence: ',
                         '(1)Name of currency borrowed at the initial time ',B,
                         '(2)Name of currency converted and invested at spot exchange rate ',A,
                         '(3)Name of currency converted at the end of the investment at the forward exchange rate',B,
                         '(4)Repayment of borrowed funds at the initial time.']
            else:
                sequence=['No arbitrage opportunity exists.']
    else:
        if M_B=='Na':
            profit=0
            sequence=['No arbitrage opportunity exists.']
        else:
            profit=0
            sequence=['No arbitrage opportunity exists.']
    return[profit,sequence]

In [15]:
value_CNY=1e8
value_USD=1.4e7
Shibor_3M=0.01594
Libor_3M=0.003129
tenor=3/12
FX_spot=7.0965
FX_forward_Jun05=7.1094

arbitrage_CNY=cov_arbitrage(S=FX_spot,F=FX_forward_Jun05,M_A=value_CNY,M_B='Na',r_A=Shibor_3M,r_B=Libor_3M,T=tenor,A='CNY',B='USD')
print('Income from borrowing CNY 100 million for covered interest arbitrage ',round(arbitrage_CNY[0],2))
print(arbitrage_CNY[1])

arbitrage_USD=cov_arbitrage(S=FX_spot,F=FX_forward_Jun05,M_A='Na',M_B=value_USD,r_A=Shibor_3M,r_B=Libor_3M,T=tenor,A='CNY',B='USD')
print('Income from borrowing USD 14 million for covered interest arbitrage ',round(arbitrage_USD[0],2))
print(arbitrage_USD[1])

Income from borrowing CNY 100 million for covered interest arbitrage  -138353.05
['No arbitrage opportunity exists.']
Income from borrowing USD 14 million for covered interest arbitrage  19334.28
['arbitrage sequence: ', '(1)Name of currency borrowed at the initial time ', 'USD', '(2)Name of currency converted and invested at spot exchange rate ', 'CNY', '(3)Name of currency converted at the end of the investment at the forward exchange rate', 'USD', '(4)Repayment of borrowed funds at the initial time.']


# 6.6.3.FX forward valuation

In [24]:
def Value_FX_Forward(F1,F2,S,par,R,t,pc,vc,position):
    '''Define a function for FX forward valuation
    F1: Forward FX rate agreed on the initial date of the contract. A/B
    F2: Forward FX rate at valuation date
    S:  Spot FX rate at valuation date
    par:Notional amount at currency pc
    R: Rf(continuous compound)of currency which is different from notional currency
    t: Remaining tenor of contract in year
    pc: par currency. 'A'indicates currency A, otherwise currency B
    vc: valuation currency. 'A'indicates currency A, otherwise currency B
    position: 'long' indicates long position, otherwise short position
    '''
    from numpy import exp
    if pc=='A':
        if position=='long':
            if vc=='A':
                value=S*(par/F2-par/F1)*exp(-R*t)
            else:
                value=(par/F2-par/F1)*exp(-R*t)
        else:
            if vc=='A':
                value=S*(par/F1-par/F2)*exp(-R*t)
            else:
                value=(par/F1-par/F2)*exp(-R*t)
    else:
        if position=='long':
            if vc=='A':
                value=(par*F2-par*F1)*exp(-R*t)
            else:
                value=(par*F2-par*F1)*exp(-R*t)/S
        else:
            if vc=='A':
                value=(par*F1-par*F2)*exp(-R*t)
            else:
                value=(par*F1-par*F2)*exp(-R*t)/S
    return value

Date       | Spot rate | 6M Shibor | 6M Libor | 3M Shibor | 3M Libor  
:------:   | :----:    | :----:    | :----:   | :----:    | :----:
2020/02/28 | 7.0066    | 2.5600%   | 1.3973%  | -         | - 
2020/05/28 | 7.1277    | -         | -        | 1.4300%   | 0.3500%

In [25]:
par_CNY=1e8
FX_spot_Feb28=7.0066
Shibor_6M_Feb28=0.0256
Libor_6M_Feb28=0.013973
T1=6/12

FX_forward_Feb28=FX_forward(spot=FX_spot_Feb28,r_A=Shibor_6M_Feb28,r_B=Libor_6M_Feb28,T=T1)
print('6M FX forward of CNY/USD at 2020/02/28 ',FX_forward_Feb28)

6M FX forward of CNY/USD at 2020/02/28  7.047050263335208


In [29]:
FX_spot_May28=7.1277
Shibor_3M_May28=0.0143
Libor_3M_May28=0.0035
rate_USD=np.log(1+Libor_3M_May28) #3M libor annual compound->continuous compound
T2=3/12

FX_forward_May28=FX_forward(spot=FX_spot_May28,r_A=Shibor_3M_May28,r_B=Libor_3M_May28,T=T2)
print('3M FX forward of CNY/USD at 2020/05/28 ',FX_forward_May28)

3M FX forward of CNY/USD at 2020/05/28  7.146927965530162


In [30]:
value_short=Value_FX_Forward(F1=FX_forward_Feb28,F2=FX_forward_May28,S=FX_spot_May28,par=par_CNY,R=rate_USD,t=T2,pc='A',vc='A',position='short')
value_long=Value_FX_Forward(F1=FX_forward_Feb28,F2=FX_forward_May28,S=FX_spot_May28,par=par_CNY,R=rate_USD,t=T2,pc='A',vc='B',position='long')
print('Value of contract for short position at 2020/05/28', round(value_short,2))
print('Value of contract for long position at 2020/05/28', round(value_long,2))

Value of contract for short position at 2020/05/28 1412250.82
Value of contract for long position at 2020/05/28 -198135.56
